In [1]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture 
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
import warnings
warnings.filterwarnings("ignore")

def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft=1126,appendEnergy = True)    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined
def train(eno):
    #path to training data
    source   = "C:\\Users\\Prashu Gupta\\Desktop\\project\\Final_dataset\\"+eno+"\\"   
    files    = [os.path.join(source,f) for f in os.listdir(source) if
             f.endswith('.wav')]
    features = np.asarray(())
    for path in files:
        path = path.strip()  #The strip() method returns a copy of the string with both leading and trailing characters removed
        # read the audio
        sr,audio = read(path)
        vector   = extract_features(audio,sr)
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        # when features of 5 files of speaker are concatenated, then do model training
    gmm = GaussianMixture(n_components = 16, max_iter = 200, covariance_type='diag',n_init = 3)
    gmm.fit(features)
    # dumping the trained gaussian model
    picklefile = "C:\\Users\\Prashu Gupta\\Desktop\\project\\Final_dataset\\Models\\"+eno+".gmm"
    pickle.dump(gmm,open( picklefile,'wb'))
    print(' modeling completed for speaker:',picklefile," with data point = ",features.shape)
train("10")

 modeling completed for speaker: C:\Users\Prashu Gupta\Desktop\project\Final_dataset\Models\10.gmm  with data point =  (1994, 40)
